In [ ]:
#@title
### YONSEI Univ. EE, KIM TAEYOON ###
import os
import PIL
import glob
import torch
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision
from torchsummary import summary
import datetime as dt

from PIL import Image

from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm

In [ ]:
#@title
def load_split_train_test(datadir, img_size=(256, 256), batchsize = 32, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(img_size),
                                      #  transforms.RandomHorizontalFlip(0.5),
                                       transforms.RandomCrop(224),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(img_size),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batchsize)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batchsize)
    return trainloader, testloader


In [ ]:
#@title

class Res(nn.Module):
    def __init__(self, num_cls = 3, pretrain=True, finetuning=True):
        super().__init__()
        self.model = models.resnet50(pretrained=pretrain)
        self.finetuning = finetuning
        if finetuning == False:
            for param in self.model.parameters():
                param.requires_grad = False
        fc = []
        fc += [nn.Linear(self.model.fc.in_features, 512)]
        fc += [nn.ReLU()]
        fc += [nn.Linear(512, 128)]
        fc += [nn.ReLU()]
        fc += [nn.Linear(128, num_cls)]
        fc += [nn.LogSoftmax(dim=1)]
        self.model.fc = nn.Sequential(*fc)

    def forward(self, x):
        out = self.model.forward(x)
        return out
    def get_prams(self):
        if self.finetuning:
            return list(self.model.parameters()) + list(self.fc.parameters())
        else:
            return self.model.fc.parameters()

class Mobile(nn.Module):
    def __init__(self, num_cls = 3, pretrain=True, finetuning=True):
        super().__init__()
        self.model = models.mobilenet_v2(pretrained=pretrain)
        self.finetuning = finetuning
        if finetuning == False:
            for param in self.model.parameters():
                param.requires_grad = False
        fc = []
        fc += [nn.Linear(1000, 512)]
        fc += [nn.ReLU()]
        fc += [nn.Linear(512, 128)]
        fc += [nn.ReLU()]
        fc += [nn.Linear(128, num_cls)]
        fc += [nn.LogSoftmax(dim=1)]
        self.model.fc = nn.Sequential(*fc)

    def forward(self, x):
        x = self.model(x)
        out = self.model.fc(x)
        return out

    def get_prams(self):
        if self.finetuning:
            return list(self.model.parameters()) + list(self.fc.parameters())
        else:
            return self.model.fc.parameters()

class Dense(nn.Module):
    def __init__(self, num_cls = 3, pretrain=True, finetuning=True):
        super().__init__()
        self.model = models.densenet161(pretrained=pretrain)
        self.finetuning = finetuning
        if finetuning == False:
            for param in self.model.parameters():
                param.requires_grad = False
        fc = []
        fc += [nn.Linear(1000, 512)]
        fc += [nn.ReLU()]
        fc += [nn.Linear(512, 128)]
        fc += [nn.ReLU()]
        fc += [nn.Linear(128, num_cls)]
        fc += [nn.LogSoftmax(dim=1)]
        self.model.fc = nn.Sequential(*fc)

    def forward(self, x):
        x = self.model(x)
        out = self.model.fc(x)
        # out = self.model.forward(x)
        return out
    def get_prams(self):
        if self.finetuning:
            return list(self.model.parameters()) + list(self.fc.parameters())
        else:
            return self.model.fc.parameters()

class Ensemble(nn.Module):
    def __init__(self, model1, model2, model3):
        super().__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3

    def forward(self, x):
        out1 = self.model1(x)
        out2 = self.model2(x)
        out3 = self.model3(x)

        out = (out1 + out2 + out3) / 3
        
        return out

    def get_prams(self):
        return self.model.parameters()

In [ ]:
## Functions ##

# Train model & Return train_loss, test_loss, test_accuracy, train_accuracy
def train(model, iter_n, learningrate):
  train_losses, test_losses, test_accuracy, train_accuracy = [], [], [], []
  model_opt = torch.optim.Adam(model.get_prams(), lr=learningrate)

  for i in range(iter_n):
    running_loss = 0
    for epoch in range(epochs):
      cor_t = 0
      lent_t = 0
      acc_t = []
      for inputs, labels in trainloader:
          inputs, labels = inputs.to(device), labels.to(device)
          model_opt.zero_grad()
          model_out = model(inputs)
          model_loss = criterion(model_out, labels)
          model_loss.backward()
          model_opt.step()
          running_loss += model_loss.item()

          ps_t = torch.exp(model_out)
          top_tp, top_tclass = ps_t.topk(1, dim=1)
          equals_t = (top_tclass == labels.view(*top_tclass.shape))
          cor_t += torch.sum(equals_t)
          lent_t += len(labels)       
          acc_t.append(cor_t/lent_t)
    
    test_loss = 0
    cor = 0
    lent = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            model_out = model(inputs)
            batch_loss = criterion(model_out, labels)
            test_loss += batch_loss.item()
            ps = torch.exp(model_out)
            top_p, top_class = ps.topk(1, dim=1)
            equals = (top_class == labels.view(*top_class.shape))
            cor += torch.sum(equals)
            lent += len(labels)
    train_losses.append(running_loss / len(trainloader))
    test_losses.append(test_loss / len(testloader))
    test_accuracy.append((cor/lent).item())
    acc_t = torch.tensor(acc_t)
    train_accuracy.append(torch.mean(acc_t).item())
    if ((i + 1) % 10 == 0):
      print(f"Iter: {i+1:3d}.. "
            f"Train loss: {running_loss/print_every:.3f}.. "
            f"Test loss: {test_loss/len(testloader):.3f}.. "
            f"Test accuracy: {cor/lent:.3f}.. "
            f"Train accuracy: {torch.mean(acc_t):.3f}"
            )
  return train_losses, test_losses, test_accuracy, train_accuracy

# Evaluate model
def eval_test(model):
  correct = 0
  length = 0
  model.eval()
  with torch.no_grad():
      for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        out = model(inputs)
        ps = torch.exp(out)
        top_p, top_class = ps.topk(1, dim=1)

        eq = (labels.view(*top_class.shape) == top_class)
        correct += torch.sum(eq)
        length += (labels.shape[0])
        
  return correct/length

# Plot board
def board(train_losses, test_losses, test_accuracy, train_accuracy):
  epoch_n = [ x for x in range(len(train_losses))]

  plt.style.use(['ggplot'])
  fig, ax1 = plt.subplots(figsize=(10,7))

  ax1.set_xlabel('epoch')
  ax1.set_ylabel('loss', color='black')
  ax1.plot(epoch_n, train_losses, color='g', label='train loss')
  ax1.tick_params(axis='y', labelcolor='black')
  ax1.plot(epoch_n, test_losses, color='grey', label='test loss')
  plt.legend(loc='lower left')

  ax2 = ax1.twinx()
  ax2.set_ylabel('accuracy', color='black')
  ax2.plot(epoch_n, test_accuracy, color='r', label='test acc')
  ax2.tick_params(axis='y', labelcolor='black')
  ax2.plot(epoch_n, train_accuracy, color='b', label='train acc')
  plt.legend(loc='lower center')

In [ ]:
### Set Models & Train ###

# Config
epochs = 1 ## Do not change!
steps = 0
running_loss = 0
print_every = 10
res_train_losses, res_test_losses, res_test_accuracy, res_train_accuracy = [], [], [], []
den_train_losses, den_test_losses, den_test_accuracy, den_train_accuracy = [], [], [], []
mob_train_losses, mob_test_losses, mob_test_accuracy, mob_train_accuracy = [], [], [], []
data_dir = "/content/drive/MyDrive/project/fashion" # Your data folder path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loader
trainloader, testloader = load_split_train_test(data_dir, valid_size=.2)

# Networks
res = Res(pretrain=True, finetuning=False)
res = res.to(device)
dense = Dense(pretrain=True, finetuning=False)
dense = dense.to(device)
mob = Mobile(pretrain=True, finetuning=False)
mob = mob.to(device)
criterion = nn.NLLLoss()

# Train
res_train_losses, res_test_losses, res_test_accuracy, res_train_accuracy = train(res, 100, 0.0001)
den_train_losses, den_test_losses, den_test_accuracy, den_train_accuracy = train(dense, 50, 0.0001)
mob_train_losses, mob_test_losses, mob_test_accuracy, mob_train_accuracy = train(mob, 50, 0.0001)

# Ensemble model
ens = Ensemble(res, dense, mob)

print("----------------------------------------------------------------------------")
print("Resnet (eval_test: {:.4f}".format(eval_test(res)) + ", max_test_acc: {:.4f})\n".format(max(res_test_accuracy)))
print("Densenet (eval_test: {:.4f}".format(eval_test(dense)) + ", max_test_acc: {:.4f})\n".format(max(den_test_accuracy)))
print("Mobile (eval_test: {:.4f}".format(eval_test(mob)) + ", max_test_acc: {:.4f})\n".format(max(mob_test_accuracy)))
print("Ensemble test_acc: {:.4f}".format(eval_test(ens).item()))
print("----------------------------------------------------------------------------")

In [ ]:
# Show training process graph

board(res_train_losses, res_test_losses, res_test_accuracy, res_train_accuracy)
board(den_train_losses, den_test_losses, den_test_accuracy, den_train_accuracy)
board(mob_train_losses, mob_test_losses, mob_test_accuracy, mob_train_accuracy)

In [ ]:
## Evaluate model & Visualize ##

correct = 0
length = 0
ens.eval()
with torch.no_grad():
    for inputs, labels in testloader:
      inputs, labels = inputs.to(device), labels.to(device)
      out = ens(inputs)
      ps = torch.exp(out)
      top_p, top_class = ps.topk(1, dim=1)

      eq = (labels.view(*top_class.shape) == top_class)
      correct += torch.sum(eq)
      length += (labels.shape[0])
      plt.figure(figsize=(15, 30))

      for i in range(len(inputs)):
        # {'formal': 0, 'hiphop': 1, 'vintage': 2}
        if (top_class[i] == 0):
          pred = "formal"
        elif (top_class[i] == 1):
          pred = "hiphop"
        else:
          pred = "vintage"
        
        if (labels[i] == 0):
          ans = "formal"
        elif (labels[i] == 1):
          ans = "hiphop"
        else:
          ans = "vintage"

        img = inputs[i].cpu()
        img = img.numpy()
        plt.subplot(8, 4, i+1)
        if (pred == ans):
          plt.title("Pred: " + pred + "\nAns: " + ans, color='black')
        else:
          plt.title("Pred: " + pred + "\nAns: " + ans, color='red')
        plt.axis('off')
        plt.imshow(np.transpose(img, (1, 2, 0)))
      
    print("Acc: {:.4f}".format(correct/length))

In [ ]:
## TSNE Modeling ##
# ***CAUTION*** Remove Augmentation: RandomCrop(224)
 
from sklearn.manifold import TSNE
import pandas as pd
from plotnine import *

first = 1
for x_data, y_data in testloader:
  if (first == 1):
    x = x_data
    y = y_data
    first = 0
  else:  
    x = torch.cat((x,x_data), dim=0)
    y = torch.cat((y,y_data), dim=0)
  
for x_data, y_data in trainloader:
  x = torch.cat((x,x_data), dim=0)
  y = torch.cat((y,y_data), dim=0)


x = x.flatten(start_dim=1, end_dim=-1)
y = y.unsqueeze(1)
t = torch.cat((x,y), dim=1)
t = t.numpy()

df = pd.DataFrame(t)
df.rename(columns = {196608: "label"}, inplace=True)

tsne = TSNE(n_components=2, verbose=1, perplexity=10, learning_rate=20, n_iter=1000)
tsne_result = tsne.fit_transform(x)

df_tsne = df.copy()
df_tsne['x-tsne'] = tsne_result[:,0] 
df_tsne['y-tsne'] = tsne_result[:,1] 
chart = ggplot( df_tsne, aes(x='x-tsne', y='y-tsne', color='label') ) \
        + geom_point(size=2, alpha=0.6) \
        + ggtitle("tSNE dimensions colored by digit") 

print(chart)